# Data Formats

## Overview

This notebook examines some of the different ways that data is organised within files. We'll call these *data formats*.

First, we're going to rule out a couple of very widely-used data formats, namely HTML and PDF. The reason is that we are interested in data that we can manipulate using computer code. In order to make this process easy, we need the data to be organised into a predictable structure. HTML documents &mdash; that is, the raw form of ordinary web pages &mdash; are generally not structured in a predictable way, which means that extracting data out of them is often complex and time-consuming. Web pages are designed to make them easy for humans to understand, rather than for computers to extract data from. PDF documents are similar, in that processing them with code is tricky and hard to do in general way. 

To sum up, HTML and PDF documents fail as *machine-readable* formats, since they cannot be imported straightforwardly into an application or computer program that deals with data.

## <a name="excel">Excel</a>

Microsoft Excel is a machine-readable format used for creating spreadsheets. Here is an example:
![](../images/messages.pdf)

Excel is frequently used by large organisations for publising tabular data. However, the data format belongs to Microsoft rather than being in the public domain, and we prefer not to use proprietary data formats. There are also issues with different versions of Excel, and differences depening on the platform on which they run (e.g., Windows vs. MacOS).

## <a name="csv">CSV</a>

CSV (short for "Comma Separated Values") is a simple data format for tables that can be read and written by any text editor. Each row of the table is represented as a line in the file, and the values of the cells in the row are separated by a comma (","). The next example shows what happens if we export the Excel data shown above into a CSV file:

In [9]:
%%bash
cat ../data/open_data/messages.csv

"James, Ewan",Arno,Re: Reminder,Let's walk instead,14/10/2015

The `pandas` library in Python is designed to make it easy to process tabular data, and we can use it display the CSV file so that it looks more like a table. In the next example, we import the library (and give it the short name `pd`), and then use it's `read_cv()` method to slurp up the CSV file.

In [13]:
import pandas as pd
table = pd.read_csv("../data/formats/messages.csv")
table

,To,From,Heading,Body,Date
0,"Arno, Ewan",James,Reminder,Cycling to Cramond today!,13/10/2015
1,"James, Ewan",Arno,Re: Reminder,Let's walk instead,14/10/2015


One issue to note is that if a value in the CSV file contains a comma, then we have to wrap that value with quote signs, as in `"James, Ewan"`.

## <a name="xml">XML</a>

XML (short for "eXtensible Markup Language") is a W3C open standard, used widely for storing data and for transferring it between applications and services.

Here is a simple example:

``` xml
<message>
    <to>James</to>
    <from>Arno</from>
    <heading>Reminder</heading>
    <body>Cycling to Cramond today!</body>
</message>
```
This is intended to self-explanatory, in the sense that we have marked-up the different parts of our data with explicit labels, or *tags*, enclosed in angle brackets. The representation is hierarchical, in the sense that we start off with a 'root' element, in this case `<message>`, which has four child elements: `<to>`, `<from>`, `<heading>` and `<body>`. In an XML document, we use start and end tags for marking up elements. For example, the start tag `<to>` has a corresponding end tag `</to>`. In XML, all elements must have a closing tag.

As you can see, XML is considerably more verbose than CSV, particularly when the data is tabular in nature. If we have more than one message in our data, then we have to come up with a higher-level root of the tree &mdash; we'll use  `<data>`. Then each row in the table has a corresponding `<message>` element. Third, we should be explicit about the fact that there are two distinct people to whom the messages are addressed. We'll represent this by having separate `<to>` elements for each of them. We've put this all into a file called `messages.xml` and the result of parsing this file (with the [lxml library](http://lxml.de/tutorial.html)) is shown below.

In [19]:
from lxml import etree
tree = etree.parse("../data/formats/messages.xml")
print(etree.tostring(top, pretty_print = True, encoding="unicode"))

<data>
  <message>
    <to>James</to>
    <to>Ewan</to>
    <from>Arno</from>
    <heading>Reminder</heading>
    <body>Cycling to Cramond today!</body>
    <date>13/10/2015</date>
  </message>
  <message>
    <to>Arno</to>
    <to>Ewan</to>
    <from>James</from>
    <heading>Re: Reminder</heading>
    <body>Let's walk instead</body>
    <date>14/10/2015</date>
  </message>
</data>



For more information, see this [W3C XML Tutorial](http://www.w3schools.com/xml/)

## <a name="json">JSON</a>

JSON ([JavaScript Object Notation](http://json.org)) is intended as format for transferring data that is easier for humans to write and read than XML. Unlike CSV and XML, JSON let's us represent lists directly using `[` and `]`. For example, the list containing the two strings `'James'` and `'Ewan'` is `['James', 'Ewan']`. 



In [25]:
import json
fp = open("../data/formats/messages.json", "w")
json.dump([
        {'to': ['James', 'Ewan'],
        'from': 'Arno',
        'heading': 'Reminder',
        'body': 'Cycling to Cramond today!',
        'date': '13/10/2015'},
         {'to': ['Arno', 'Ewan'],
        'from': 'James',
        'heading': 'Re: Reminder',
        'body': 'Let\'s walk instead',
        'date': '14/10/2015'},
    ], fp)

In [26]:
fp = open("../data/formats/messages.json")
data = json.load(fp)
print(json.dumps(data, indent=2, sort_keys=True))

[
  {
    "body": "Cycling to Cramond today!",
    "date": "13/10/2015",
    "from": "Arno",
    "heading": "Reminder",
    "to": [
      "James",
      "Ewan"
    ]
  },
  {
    "body": "Let's walk instead",
    "date": "14/10/2015",
    "from": "James",
    "heading": "Re: Reminder",
    "to": [
      "Arno",
      "Ewan"
    ]
  }
]
